In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch as torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models

In [2]:
import pickle as pkl
from tqdm.notebook import tqdm

In [3]:
DEVICE = torch.device("cuda:0")
DIM = 128
IMAGE_SIZE = 112
dtype = torch.cuda.FloatTensor

In [4]:
torch.cuda.is_available()

True

In [5]:
data_ = pkl.load(open('Data_siamese.pickle', 'rb'))

In [6]:
data_.shape

(6317, 20, 112, 112, 3)

In [7]:
data = data_[:4000, :, :, :, :]

In [8]:
model = models.mobilenet_v2(width_mult=1, pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))

In [9]:
class Bottleneck(nn.Module):
    def __init__(self, model, in_dim=1280, out_dim=DIM, spartial=(IMAGE_SIZE+31) // 32):
        super().__init__()
        self.model = model
        self.depthwise = nn.Conv2d(in_dim, in_dim, spartial, bias=False, groups=in_dim)
        self.linear = nn.Linear(in_dim, out_dim)
        
    def forward(self, x):
        x = self.model(x)
        x = self.depthwise(x)
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        return x

In [10]:
model = Bottleneck(model)
model = model.to(DEVICE)

In [12]:
class Probcalc(nn.Module):
    def __init__(self, hidden_dim=32):
        super().__init__()
        self.inp = nn.Linear(1, hidden_dim)
        self.linear1 = nn.Linear(hidden_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = x.unsqueeze(-1)
        x = self.inp(x)
        x = F.leaky_relu(x, 0.1)
        x = self.linear1(x)
        x = F.leaky_relu(x, 0.1)
        x = self.linear2(x)
        x = F.leaky_relu(x, 0.1)
        x = self.linear3(x)
        x = F.leaky_relu(x, 0.1)
        x = self.out(x)
        x = x ** 2
        x = x.squeeze(-1)
        return torch.exp(-x)

In [13]:
calcprob = Probcalc().to(DEVICE)

In [14]:
optimizer = torch.optim.Adam(list(model.parameters()) + list(calcprob.parameters()), lr=0.003)

In [15]:
def change_lr(optimizer, epoch):
    lr = 1e-3
    if epoch < 20:
        lr = 1e-3
    elif epoch < 30:
        lr = 1e-3 / 2.0
    else:
        lr = 1e-4
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [16]:
rng = np.random.default_rng()

In [17]:
import cv2
from PIL import Image

In [18]:
def dist(x1, x2):
    return torch.abs(x1 - x2)

In [19]:
def d(x, y):
  # Cosine similarity
  return 1 - (x * y).sum(dim=-1) / (torch.sqrt((x * x).sum(dim=-1)) * torch.sqrt((y * y).sum(dim=-1)))

In [20]:
EPOCHS = 60
STEP_PER_EPOCH = 2000
BATCH = 128

In [21]:
def gen_pair():
    batch_data_1 = torch.zeros(BATCH, 3, 112, 112).byte()
    batch_data_2 = torch.zeros(BATCH, 3, 112, 112).byte()
    batch_target = torch.zeros(BATCH, 1)
    bad = rng.choice(4000, size = BATCH, replace=False)
    good = rng.choice(4000, size = BATCH // 2, replace=True)
    for i in range(BATCH):
        if (i % 2 == 0):
            ind = rng.choice(20, size=2, replace=False)
            batch_data_1[i] = torch.tensor((data[good[i // 2], ind[0]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_data_2[i] = torch.tensor((data[good[i // 2], ind[1]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_target[i] = 1
        else:
            ind = rng.choice(20, size=2, replace=True)
            batch_data_1[i] = torch.tensor((data[bad[i // 2], ind[0]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_data_2[i] = torch.tensor((data[bad[i // 2 + BATCH // 2], ind[1]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_target[i, 0] = 0
    return batch_data_1, batch_data_2, batch_target

In [22]:
from torchvision import transforms

In [23]:
for epoch in range(EPOCHS):
    avg_loss = 0.0
    for s in tqdm(range(STEP_PER_EPOCH)):
        change_lr(optimizer, epoch)
        optimizer.zero_grad()        
        X1, X2, target = gen_pair()
        X1 = X1.to(DEVICE).float()
        X2 = X2.to(DEVICE).float()
        target = target.to(DEVICE).float()
        diff = d(model(X1), model(X2))
        prob = calcprob(diff)
        loss = F.binary_cross_entropy(prob.unsqueeze(-1), target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(list(model.parameters()) + list(calcprob.parameters()), 10)
        optimizer.step()
        avg_loss += loss.item()
        
    print("Train epoch", epoch, "finished with avg_loss", avg_loss / STEP_PER_EPOCH)
    avg_loss = 0
    avg_acc = 0


Train epoch 0 finished with avg_loss 0.7005546538829803



Train epoch 1 finished with avg_loss 0.6869382807612419



Train epoch 2 finished with avg_loss 0.6833848099112511



Train epoch 3 finished with avg_loss 0.6813656768798828



Train epoch 4 finished with avg_loss 0.6776510311365127


...


Train epoch 55 finished with avg_loss 0.5758484231978654



Train epoch 56 finished with avg_loss 0.5745375305116177



Train epoch 57 finished with avg_loss 0.5734826191365718



Train epoch 58 finished with avg_loss 0.5718943717181683



Train epoch 59 finished with avg_loss 0.572985222786665


In [24]:
torch.save(model.state_dict(), "model_ver2.state")
torch.save(calcprob.state_dict(), "dist2prob_ver2.state")

In [29]:
BATCH = 4

In [30]:
def gen_pair_test(true_pair = True):
    batch_data_1 = torch.zeros(BATCH, 3, 112, 112).byte()
    batch_data_2 = torch.zeros(BATCH, 3, 112, 112).byte()
    if true_pair:
        good = rng.choice(2300, size = BATCH, replace=True)
        for i in range(BATCH):
            ind = rng.choice(20, size=2, replace=True)
            batch_data_1[i] = torch.tensor((data[good[i], ind[0]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_data_2[i] = torch.tensor((data[good[i], ind[1]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
    else:
        for i in range(BATCH):
            ind1 = rng.choice(2300, size=2, replace=False)
            ind = rng.choice(20, size=2, replace=True)
            batch_data_1[i] = torch.tensor((data[ind1[0], ind[0]] - 255 / 2) /  (255 / 2)).transpose(0, 2)
            batch_data_2[i] = torch.tensor((data[ind1[1], ind[1]] - 255 / 2) /  (255 / 2)).transpose(0, 2)   
        
    return batch_data_1, batch_data_2

In [32]:
cnt = 0.0
for s in tqdm(range(2000)):  
        X1, X2 = gen_pair_test()
        X1 = X1.to(DEVICE).float()
        X2 = X2.to(DEVICE).float()
        diff = d(model(X1), model(X2))
        prob = calcprob(diff)
        cnt += (prob >= 0.5).sum().item()
        
print("Accurancy on true pairs ", cnt / (2000 * BATCH))

cnt1 = 0.0
for s in tqdm(range(2000)):  
        X1, X2 = gen_pair_test(true_pair = False)
        X1 = X1.to(DEVICE).float()
        X2 = X2.to(DEVICE).float()
        diff = d(model(X1), model(X2))
        prob = calcprob(diff)
        cnt1 += (prob < 0.5).sum().item()
        
        
print("Accurancy on false pairs ", cnt1 / (2000 * BATCH))


Accurancy on true pairs  0.6665



Accurancy on false pairs  0.5255
